In [1]:
from datasets import load_dataset
dataset = load_dataset("shunk031/JGLUE", 'JCommonsenseQA')

Found cached dataset jglue (/home/lan/.cache/huggingface/datasets/shunk031___jglue/JCommonsenseQA/1.1.0/3bec28582f7c64bc25fb042e3fffdbcd04c45bce4e494a6c6baa8ccdc53185ca)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def build_prompt(row):
    return f"""以下の問いを読んで、A-D選択肢からあてはまるものを一つ選び、回答してください：

# 質問:
主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？

A. 世界
B. 写真集
C. 絵本
D. 論文
E. 図鑑

# 回答:
C

#  質問:
未成年者を監護・教育し，彼らを監督し，彼らの財産上の利益を守る法律上の義務をもつ人は？

A. 浮浪者
B. 保護者
C. お坊さん
D. 宗教者
E. 預言者

# 回答:
B

#  質問:
数字の１を表すときに使う体は？

A. 胸
B. 肉球
C. 背中
D. 人差し指
E. 親指

# 回答:
D

#  質問:
{row['question']}

A. {row['choice0']}
B. {row['choice1']}
C. {row['choice2']}
D. {row['choice3']}
E. {row['choice4']}

# 回答:
"""

print(build_prompt(dataset['train'][0]))

以下の問いを読んで、A-D選択肢からあてはまるものを一つ選び、回答してください：

# 質問:
主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？

A. 世界
B. 写真集
C. 絵本
D. 論文
E. 図鑑

# 回答:
C

#  質問:
未成年者を監護・教育し，彼らを監督し，彼らの財産上の利益を守る法律上の義務をもつ人は？

A. 浮浪者
B. 保護者
C. お坊さん
D. 宗教者
E. 預言者

# 回答:
B

#  質問:
数字の１を表すときに使う体は？

A. 胸
B. 肉球
C. 背中
D. 人差し指
E. 親指

# 回答:
D

#  質問:
主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？

A. 世界
B. 写真集
C. 絵本
D. 論文
E. 図鑑

# 回答:



In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import logging

def load_model(name_or_path='/mnt/h/git/llama/llama-2-7b-hf',
               model_type='huggingface',
               device='cuda:0'):
    tokenizer = AutoTokenizer.from_pretrained(name_or_path, trust_remote_code=True)
    match model_type:
        case 'gptq':
            from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
            model = AutoGPTQForCausalLM.from_quantized(name_or_path,
                                               use_safetensors=True,
                                               device=device)
        case _:
            from transformers import AutoModelForCausalLM
            model = AutoModelForCausalLM.from_pretrained(name_or_path, torch_dtype=torch.float16).to(device)
    return tokenizer, model

tokenizer, model = load_model()

def answer(prompt):
    return tokenizer.decode(
        model.generate(**tokenizer(prompt, return_tensors="pt").to(model.device),
                       temperature=0.1,
                       max_new_tokens=3)[0])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
mappings = {'A':0,'B':1,'C':2,'D':3,'E':4}

def run_test(item, verbose=False):
    res = answer(build_prompt(item))

    if verbose:
        print(item)
        print(res)

    res = res.split('# 回答:')[-1].strip()
    
    if res in mappings:
        final_answer = mappings[res]
    else:
        print(f'Cannot parse answer<{res}> for {item}')
        final_answer = None

    if final_answer == item['label']:
        return 1
    else:
        print('wrong sample', item, res)
        return 0

run_test(dataset['validation'][0], verbose=True)

{'q_id': 8939, 'question': '電子機器で使用される最も主要な電子回路基板の事をなんと言う？', 'choice0': '掲示板', 'choice1': 'パソコン', 'choice2': 'マザーボード', 'choice3': 'ハードディスク', 'choice4': 'まな板', 'label': 2}
<s> 以下の問いを読んで、A-D選択肢からあてはまるものを一つ選び、回答してください：

# 質問:
主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？

A. 世界
B. 写真集
C. 絵本
D. 論文
E. 図鑑

# 回答:
C

#  質問:
未成年者を監護・教育し，彼らを監督し，彼らの財産上の利益を守る法律上の義務をもつ人は？

A. 浮浪者
B. 保護者
C. お坊さん
D. 宗教者
E. 預言者

# 回答:
B

#  質問:
数字の１を表すときに使う体は？

A. 胸
B. 肉球
C. 背中
D. 人差し指
E. 親指

# 回答:
D

#  質問:
電子機器で使用される最も主要な電子回路基板の事をなんと言う？

A. 掲示板
B. パソコン
C. マザーボード
D. ハードディスク
E. まな板

# 回答:
C




1

In [7]:
import tqdm

test_ds = dataset['validation']
results = []

for i in tqdm.notebook.tqdm(range(len(test_ds))):
    results.append(run_test(test_ds[i]))


  0%|          | 0/1119 [00:00<?, ?it/s]

wrong sample {'q_id': 8941, 'question': 'しゃがんだりする様を何という？', 'choice0': '腰を下す', 'choice1': '座る', 'choice2': '仮眠を取る', 'choice3': '寝る', 'choice4': '起きる', 'label': 0} D
wrong sample {'q_id': 8942, 'question': '水を出すときに捻るものは？', 'choice0': '蛇口', 'choice1': 'ハンドル', 'choice2': '流し', 'choice3': '釘', 'choice4': '食器棚', 'label': 0} B
wrong sample {'q_id': 8944, 'question': '地中にある一定の大きさの空間のこと？', 'choice0': '麓', 'choice1': '山頂', 'choice2': '中腹', 'choice3': '山腹', 'choice4': '洞窟', 'label': 4} D
wrong sample {'q_id': 8945, 'question': '乗り物が関係する言葉はどれ？', 'choice0': '外に出る', 'choice1': '歩く', 'choice2': '靴を脱ぐ', 'choice3': '靴を履く', 'choice4': '車から降りる', 'label': 4} D
wrong sample {'q_id': 8946, 'question': '持ち運び、使いまわりの良さに富んだパソコンをなんという？', 'choice0': 'コンピュータ', 'choice1': 'ロープ', 'choice2': 'ノートパソコン', 'choice3': '携帯', 'choice4': '軽食', 'label': 2} A
wrong sample {'q_id': 8948, 'question': 'スピードを超過して警察に捕まることといえば？', 'choice0': '妨害運転', 'choice1': '悲しい', 'choice2': 'かさぶたができる', 'choice3': '危険運転', 'choice4': '交通違反', 'label

In [8]:
total = len(results)
correct = sum(results)

print(f'Correct: {correct}/{total}, Accuracy: {correct/total}')

Correct: 771/1119, Accuracy: 0.6890080428954424
